In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder \
    .appName("transform_population") \
    .master("local[*]") \
    .getOrCreate()

In [62]:
df = spark.read \
    .options(header=True, sep=r'\t') \
    .csv("population")
df.show()

+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
| indic_de,geo\time| 2008 |2009 | 2010 | 2011 | 2012 |2013 | 2014 | 2015 |2016 | 2017 | 2018 | 2019 |
+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
|       PC_Y0_14,AD| 14.6 |14.5 | 14.5 | 15.5 | 15.5 |15.5 |    : |    : |   : |    : |    : | 13.9 |
|       PC_Y0_14,AL| 24.1 |23.3 | 22.5 | 21.6 | 20.7 |20.1 | 19.6 | 19.0 |18.5 | 18.2 | 17.7 | 17.2 |
|       PC_Y0_14,AM| 19.0 |18.6 | 18.3 |    : |    : |   : |    : | 19.4 |19.6 | 20.0 | 20.2 | 20.2 |
|       PC_Y0_14,AT| 15.4 |15.1 | 14.9 | 14.7 | 14.6 |14.4 | 14.3 | 14.3 |14.3 | 14.4 | 14.4 | 14.4 |
|       PC_Y0_14,AZ| 23.2 |22.6 | 22.6 | 22.3 | 22.2 |22.3 | 22.4 | 22.4 |22.5 | 22.6 | 22.6 | 22.4 |
|       PC_Y0_14,BE| 16.9 |16.9 | 16.9 |17.0 b| 17.0 |17.0 | 17.0 | 17.0 |17.0 | 17.0 | 17.0 | 16.9 |
|       PC_Y0_14,BG| 13.1 |13.1 | 13.2 | 13.2 | 13.4 |13.6 | 13.7 | 13.9 |14.0 | 1

In [63]:
df.show()

+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
| indic_de,geo\time| 2008 |2009 | 2010 | 2011 | 2012 |2013 | 2014 | 2015 |2016 | 2017 | 2018 | 2019 |
+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
|       PC_Y0_14,AD| 14.6 |14.5 | 14.5 | 15.5 | 15.5 |15.5 |    : |    : |   : |    : |    : | 13.9 |
|       PC_Y0_14,AL| 24.1 |23.3 | 22.5 | 21.6 | 20.7 |20.1 | 19.6 | 19.0 |18.5 | 18.2 | 17.7 | 17.2 |
|       PC_Y0_14,AM| 19.0 |18.6 | 18.3 |    : |    : |   : |    : | 19.4 |19.6 | 20.0 | 20.2 | 20.2 |
|       PC_Y0_14,AT| 15.4 |15.1 | 14.9 | 14.7 | 14.6 |14.4 | 14.3 | 14.3 |14.3 | 14.4 | 14.4 | 14.4 |
|       PC_Y0_14,AZ| 23.2 |22.6 | 22.6 | 22.3 | 22.2 |22.3 | 22.4 | 22.4 |22.5 | 22.6 | 22.6 | 22.4 |
|       PC_Y0_14,BE| 16.9 |16.9 | 16.9 |17.0 b| 17.0 |17.0 | 17.0 | 17.0 |17.0 | 17.0 | 17.0 | 16.9 |
|       PC_Y0_14,BG| 13.1 |13.1 | 13.2 | 13.2 | 13.4 |13.6 | 13.7 | 13.9 |14.0 | 1

In [64]:
df1 = df.select(f.col(r'indic_de,geo\time').alias('code'), f.col('2019 ').alias('Y2019')) \
    .withColumn('country_code', f.split('code', ',')[1]) \
    .withColumn('ages_group', f.split((f.split('code', ',')[0]), 'Y')[1])

df1.show(50)

+------------------+------+------------+----------+
|              code| Y2019|country_code|ages_group|
+------------------+------+------------+----------+
|       PC_Y0_14,AD| 13.9 |          AD|      0_14|
|       PC_Y0_14,AL| 17.2 |          AL|      0_14|
|       PC_Y0_14,AM| 20.2 |          AM|      0_14|
|       PC_Y0_14,AT| 14.4 |          AT|      0_14|
|       PC_Y0_14,AZ| 22.4 |          AZ|      0_14|
|       PC_Y0_14,BE| 16.9 |          BE|      0_14|
|       PC_Y0_14,BG| 14.4 |          BG|      0_14|
|       PC_Y0_14,BY| 16.9 |          BY|      0_14|
|       PC_Y0_14,CH| 15.0 |          CH|      0_14|
|       PC_Y0_14,CY| 16.1 |          CY|      0_14|
|       PC_Y0_14,CZ| 15.9 |          CZ|      0_14|
|       PC_Y0_14,DE| 13.6 |          DE|      0_14|
|       PC_Y0_14,DK| 16.5 |          DK|      0_14|
|     PC_Y0_14,EA18|15.0 p|        EA18|      0_14|
|     PC_Y0_14,EA19|15.0 p|        EA19|      0_14|
|       PC_Y0_14,EE| 16.4 |          EE|      0_14|
|       PC_Y

In [65]:
df1.createOrReplaceTempView('population')


In [66]:
spark.sql("""
SELECT distinct ages_group
FROM population
""").show()

+----------+
|ages_group|
+----------+
|     65_79|
|      0_14|
|     50_64|
|     15_24|
|    80_MAX|
|     25_49|
+----------+



In [67]:
df_population_pivot = spark.sql("""
SELECT cast(regexp_replace(Y2019, '[a-z]', '') as decimal(10,2) ) Y2019, country_code, ages_group
FROM population
WHERE length(country_code) == 2 
""").groupBy('country_code').pivot('ages_group').max('Y2019') \
    .withColumn('sum', f.col('0_14')+f.col('15_24')+f.col('25_49')+f.col('50_64')+f.col('65_79')+f.col('80_MAX')) \
    .orderBy('country_code')


df_population_pivot.show(100)

+------------+-----+-----+-----+-----+-----+------+------+
|country_code| 0_14|15_24|25_49|50_64|65_79|80_MAX|   sum|
+------------+-----+-----+-----+-----+-----+------+------+
|          AD|13.90|10.60|39.40|22.50|10.20|  3.40|100.00|
|          AL|17.20|15.50|33.00|20.20|11.40|  2.70|100.00|
|          AM|20.20|11.80|36.90|19.10| 9.00|  3.00|100.00|
|          AT|14.40|10.90|34.00|21.70|13.80|  5.00| 99.80|
|          AZ|22.40|14.10|39.10|17.60| 5.30|  1.50|100.00|
|          BE|16.90|11.40|32.70|20.10|13.30|  5.60|100.00|
|          BG|14.40| 8.90|35.00|20.40|16.50|  4.80|100.00|
|          BY|16.90| 9.90|36.60|21.30|11.30|  3.90| 99.90|
|          CH|15.00|10.60|35.00|20.90|13.30|  5.20|100.00|
|          CY|16.10|12.80|37.10|17.90|12.50|  3.70|100.10|
|          CZ|15.90| 9.10|36.70|18.80|15.50|  4.10|100.10|
|          DE|13.60|10.40|31.60|22.80|15.10|  6.50|100.00|
|          DK|16.50|12.60|31.70|19.60|15.00|  4.50| 99.90|
|          EE|16.40| 9.50|34.90|19.40|14.10|  5.60| 99.9

In [69]:
df_population_pivot.createOrReplaceTempView('population_pivot')

In [70]:
df_country_lookup = spark.read \
    .options(header=True, sep=',') \
    .csv('dim_country/')
df_country_lookup.show()
df_country_lookup.createOrReplaceTempView('country_lookup')

+--------------------+--------------------+--------------------+---------+----------+
|             country|country_code_2_digit|country_code_3_digit|continent|population|
+--------------------+--------------------+--------------------+---------+----------+
|               Aruba|                  AW|                 ABW|  America|    106766|
|         Afghanistan|                  AF|                 AFG|     Asia|  38928341|
|              Angola|                  AO|                 AGO|   Africa|  32866268|
|            Anguilla|                  AI|                 AIA|  America|     15002|
|             Albania|                  AL|                 ALB|   Europe|   2862427|
|             Andorra|                  AD|                 AND|   Europe|     76177|
|United Arab Emirates|                  AE|                 ARE|     Asia|   9890400|
|           Argentina|                  AR|                 ARG|  America|  45195777|
|             Armenia|                  AM|           

In [72]:
df_processed_population = spark.sql("""
SELECT c.country,
       c.country_code_2_digit,
       c.country_code_3_digit,
       c.population,
       p.0_14  AS age_group_0_14,
       p.15_24 AS age_group_15_24,
       p.25_49 AS age_group_25_49,
       p.50_64 AS age_group_50_64, 
       p.65_79 AS age_group_65_79,
       p.80_MAX AS age_group_80_max
FROM population_pivot p JOIN country_lookup c ON p.country_code = c.country_code_2_digit
ORDER BY c.country
""")
df_processed_population.show(truncate=False)

+----------+--------------------+--------------------+----------+--------------+---------------+---------------+---------------+---------------+----------------+
|country   |country_code_2_digit|country_code_3_digit|population|age_group_0_14|age_group_15_24|age_group_25_49|age_group_50_64|age_group_65_79|age_group_80_max|
+----------+--------------------+--------------------+----------+--------------+---------------+---------------+---------------+---------------+----------------+
|Albania   |AL                  |ALB                 |2862427   |17.20         |15.50          |33.00          |20.20          |11.40          |2.70            |
|Andorra   |AD                  |AND                 |76177     |13.90         |10.60          |39.40          |22.50          |10.20          |3.40            |
|Armenia   |AM                  |ARM                 |2963234   |20.20         |11.80          |36.90          |19.10          |9.00           |3.00            |
|Austria   |AT              